# Traffic Lights Route Planner

Use this notebook to plan a route and visualize traffic lights along the way. Enter start and end addresses below.

In [ ]:
import requests
import folium
from geopy.geocoders import Nominatim
from ipywidgets import Text, Button, VBox, Output
from IPython.display import display

geolocator = Nominatim(user_agent='traffic_lights_route')

def geocode(address: str):
    location = geolocator.geocode(address)
    if not location:
        raise ValueError(f'Could not geocode address: {address}')
    return (location.latitude, location.longitude)

def get_route(start, end):
    url = f"https://router.project-osrm.org/route/v1/driving/{start[1]},{start[0]};{end[1]},{end[0]}?overview=full&geometries=geojson"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()
    coords = data['routes'][0]['geometry']['coordinates']
    return [(lat, lon) for lon, lat in coords]

def fetch_traffic_lights(route, padding=0.005):
    lats = [lat for lat, _ in route]
    lons = [lon for _, lon in route]
    min_lat, max_lat = min(lats) - padding, max(lats) + padding
    min_lon, max_lon = min(lons) - padding, max(lons) + padding
    query = (
        f"[out:json];"
        f"node['highway'='traffic_signals']({min_lat},{min_lon},{max_lat},{max_lon});"
        "out;"
    )
    r = requests.get("https://overpass-api.de/api/interpreter", params={'data': query})
    r.raise_for_status()
    data = r.json()
    lights = []
    for el in data.get('elements', []):
        lat = el['lat']
        lon = el['lon']
        name = el.get('tags', {}).get('name', 'Traffic Light')
        lights.append((lat, lon, name))
    return lights

start_box = Text(description='Start:')
end_box = Text(description='End:')
button = Button(description='Show Route')
output = Output()

def on_click(_):
    output.clear_output()
    with output:
        start = geocode(start_box.value)
        end = geocode(end_box.value)
        route = get_route(start, end)
        lights = fetch_traffic_lights(route)
        m = folium.Map(location=start, zoom_start=13)
        folium.PolyLine(route, color='blue', weight=5).add_to(m)
        for lat, lon, name in lights:
            folium.Marker((lat, lon), popup=name, icon=folium.Icon(color='red', icon='traffic-light')).add_to(m)
        display(m)

button.on_click(on_click)

VBox([start_box, end_box, button, output])
